## 04 — Recommendation Articles (Task 3)
Goal: turn the meta-categories + clusters into short “recommendation-style” writeups.

Outputs:
- `reports/artifacts/recommendation_articles.md`
- (optional) `reports/artifacts/recommendation_articles.json`

In [1]:
import pandas as pd
from pathlib import Path

dash = pd.read_csv("../reports/artifacts/theme_dashboard_with_meta.csv")
dash.head(3)

,sentiment,cluster_id,size,top_phrases,theme_summary,rep_1,rep_2,rep_3,meta_category
0,negative,0,80,"tablet, amazon, just, kindle, use, apps","Negative reviews mention: tablet, amazon, just...",I do not like this Tablet one bit. It has to b...,This product is real bad from amazon. Although...,I had an old ipad and when it quit I decided t...,Setup/usability
1,negative,1,5,"slow, slow returning, suggestions, returning d...","Negative reviews mention: slow, slow returning...",Barely 4k content and it lags a lot. Not worth...,"Super slow, not of decent quality.I would not ...",So slow returning mine for something different...,Customer service/returns
2,neutral,0,97,"tablet, good, amazon, ok, apps, kindle","Neutral reviews mention: tablet, good, amazon,...",This was a good tablet. I bought it for my Gf ...,"This is a good tablet for the price, however, ...","Bought this for my nephew. The price is right,...",Setup/usability


## Quick check
Make sure the meta categories are present and not all "Other".

In [2]:
dash["meta_category"].value_counts()

meta_category
Setup/usability              3
Other                        2
Compatibility/accessories    2
Customer service/returns     1
Build quality                1
Delivery/packaging           1
Name: count, dtype: int64

## Build one article per meta-category
We’ll use:
- biggest positive themes (what people like)
- biggest negative themes (what people complain about)
- a couple representative reviews as examples

In [3]:
def short(s, n=240):
    s = str(s).replace("\n", " ").strip()
    return s if len(s) <= n else s[:n].rstrip() + "…"

def top(df, n=3):
    return df.sort_values("size", ascending=False).head(n)

def make_article(cat, d):
    pos = d[d["sentiment"] == "positive"]
    neg = d[d["sentiment"] == "negative"]

    lines = []
    lines.append(f"## {cat}\n")

    lines.append("### What people like")
    if len(pos) == 0:
        lines.append("- (no clear positive theme yet)")
    else:
        for _, r in top(pos, 3).iterrows():
            lines.append(f"- ({int(r['size'])} reviews) {short(r.get('theme_summary',''))}")

    lines.append("\n### Common complaints")
    if len(neg) == 0:
        lines.append("- (no clear complaint theme yet)")
    else:
        for _, r in top(neg, 3).iterrows():
            lines.append(f"- ({int(r['size'])} reviews) {short(r.get('theme_summary',''))}")

    lines.append("\n### Example review")
    r0 = top(d, 1).iloc[0]
    ex = r0.get("rep_1", "")
    lines.append(f"- “{short(ex, 280)}”" if isinstance(ex, str) and ex.strip() else "- (no example available)")

    lines.append("\n---\n")
    return "\n".join(lines)

## Generate articles for each meta-category
We’ll create one section per category (and skip "Other" for now).

In [4]:
cats = [c for c in dash["meta_category"].dropna().unique().tolist() if c != "Other"]
cats

['Setup/usability',
 'Customer service/returns',
 'Build quality',
 'Compatibility/accessories',
 'Delivery/packaging']

## Build the markdown file
This creates a single markdown document with all the category writeups.

In [5]:
articles = ["# Recommendation Articles\n"]

for cat in cats:
    d_cat = dash[dash["meta_category"] == cat].copy()
    articles.append(make_article(cat, d_cat))

final_md = "\n".join(articles)
final_md[:1500]

'# Recommendation Articles\n\n## Setup/usability\n\n### What people like\n- (24 reviews) Positive reviews mention: easy, use, easy use, set, easy set, price.\n\n### Common complaints\n- (80 reviews) Negative reviews mention: tablet, amazon, just, kindle, use, apps.\n\n### Example review\n- “This was a good tablet. I bought it for my Gf who basically only wanted it for Netflix and YouTube. The display was good, not great. The issue we had was that it kept dropping the wifi connection. This could have been a defect in this one tablet but we returned it and got another…”\n\n---\n\n## Customer service/returns\n\n### What people like\n- (no clear positive theme yet)\n\n### Common complaints\n- (5 reviews) Negative reviews mention: slow, slow returning, suggestions, returning different, returning, different suggestions.\n\n### Example review\n- “Barely 4k content and it lags a lot. Not worth it. Returned!”\n\n---\n\n## Build quality\n\n### What people like\n- (988 reviews) Positive reviews m

## Save to reports/artifacts

In [6]:
from pathlib import Path

out_md = Path("../reports/artifacts/recommendation_articles.md")
out_md.parent.mkdir(parents=True, exist_ok=True)
out_md.write_text(final_md, encoding="utf-8")

out_md.as_posix()

'../reports/artifacts/recommendation_articles.md'

## Quick preview
Just to make sure the file content looks good.

In [7]:
print(final_md[:2500])

# Recommendation Articles

## Setup/usability

### What people like
- (24 reviews) Positive reviews mention: easy, use, easy use, set, easy set, price.

### Common complaints
- (80 reviews) Negative reviews mention: tablet, amazon, just, kindle, use, apps.

### Example review
- “This was a good tablet. I bought it for my Gf who basically only wanted it for Netflix and YouTube. The display was good, not great. The issue we had was that it kept dropping the wifi connection. This could have been a defect in this one tablet but we returned it and got another…”

---

## Customer service/returns

### What people like
- (no clear positive theme yet)

### Common complaints
- (5 reviews) Negative reviews mention: slow, slow returning, suggestions, returning different, returning, different suggestions.

### Example review
- “Barely 4k content and it lags a lot. Not worth it. Returned!”

---

## Build quality

### What people like
- (988 reviews) Positive reviews mention: tablet, great, kindle, p

## Save JSON for the web app
This makes it easy to show an article from a dropdown later.

In [8]:
import json
from pathlib import Path

out_json = Path("../reports/artifacts/recommendation_articles.json")
payload = {cat: make_article(cat, dash[dash["meta_category"] == cat]) for cat in cats}
out_json.write_text(json.dumps(payload, indent=2), encoding="utf-8")

out_json.as_posix()

'../reports/artifacts/recommendation_articles.json'